# TP3 - Market Basket Analysis 
INF8111 - Fouille de données, Fall 2019
### Team Components
    - Member 1
    - Member 2
    - Member 3

**Authors**: Rodrigo Randel

## Date et directives de remise
Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **3 decembre à 23h55**.

## Market Basket Analysis

Market Basket Analysis (MBA) is a data mining analytics technique to uncover associations between products or product grouping. By exploring interesting patterns from an extensive collection of data, MBA aims to understand/reveal customer purchase behaviors based upon the theory that if you purchased a certain set of products, then you are more (or less) likely to buy another group of products. In other words, MBA allows retailers to identify the relationship between the items that customers buy, revealing patterns of items often purchased together.

A widely approach to explore these patterns is by constructing ***association rules*** such as:
- **if** bought *ITEM_1* **then** will buy *ITEM_2* with **confidence** *X*.

These associations do not have to be 1-to-1 rules. They can involve many items. For example, a person in a supermarket may add eggs to his/her cart, then an MBA application may suggest that the person will also buy some bread and/or flour: 
    
+ **if** bought *EGGS* **then** will buy [*BREAD* with confidence *0.2*; *FLOUR* with confidence 0.05].

However, if the person now decides to add flour to his/her cart, the new association rule could be as showing below, suggesting ingredients to make a cake.

+ **if** bought [*EGGS, FLOUR*] **then** will buy [*SUGGAR* with confidence 0.45; BAKING POWDER with confidence 0.12; *BREAD* with confidence *0.03*].


There are many real scenarios where MBA plays a central role in data analysis, such as supermarket transactions, online orders or credit card history. Marketers may use these association rules to allocate correlated products close to each other on store shelves or make online suggestions so that customers buy more items. Some questions that an MBA can usually help retailers to answer are:

    - What items are often purchased together?
    - Given a basket, what items should be suggested?
    - How should items be placed together on the shelves?


### Objective

Your goal in this TP is to develop an MBA algorithm for revealing patterns by creating association rules in a big dataset with more than three millions supermarket transactions. However, mining association rules for large datasets is a very computationally intensive problem, which makes it almost impractical to perform it without a distributed system. Hence, to run your algorithm, you will have access to a distributed cloud computing cluster with hundreds of cores. 

To this end, a **MapReduce** algorithm will be implemented upon the [Apache Spark](http://spark.apache.org) framework, a fast cluster computing system. In a nutshell, Spark is an open source framework designed with a *scale-out* methodology which makes it a very powerful tool for programmers or application developers to perform a massive volume of computations and data processing in distributed environments. Sparks provides high-level APIs that make it easy to build parallel apps without needing to worry about how your code and data are parallelized/distributed thought the computing cluster. Spark does it all for you.
 
The implementation will follow the Market Basket Analysis algorithm presented by Jongwook Woo and Yuhang Xu (2012). The image **workflow.svg** Illustrates the algorithm's workflow, and is to be used for consultation throughout this TP. The blue boxes are the ones where you must implement a method to perform a map or reduce function, and the grey boxes represent their expected output. **All these operations are explained in details in the following sections.** 

<!---
# ![scale=0.5](workflow.svg "Algorithm Workflow")
-->

## 1. Setting up Spark

Spark runs on both Windows and UNIX-like systems (e.g., Linux, Mac OS). It's easy to run locally on one machine — all you need is to have Java installed on your system PATH, or the JAVA_HOME environment variable pointing to a Java installation. It is recommended that you have the **JDK v8** installed in your system. If you haven't, go to [Java's web page](https://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html) to download and install a Java Virtual Machine. Remember to set the environment variable JAVA_HOME if your installation does not do it automatically for you. 

The interface between Python and Spark is done through **PySpark**, which can be obtained following the sequence below:

1. First, go to http://spark.apache.org/downloads 
2. Select the newest Spark release and the Pre-built for Apache Hadoop 2.7 package 
3. Click for download **spark-2.4.4-bin-hadoop2.7.tgz** and unzip it in any folder of your preference. 
4. Next, export the following variables to link PYSPARK (Spark's python interface) to your python distribution in yout `~/.bash_profile` file.
```
    - export SPARK_HOME=/path/to/spark-2.4.4-bin-hadoop2.7
    - export PYTHONPATH="$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.7-src.zip:$SPARK_HOME/python/lib/pyspark.zip:$PYTHONPATH"
    - export PYSPARK_PYTHON=/path/to/your/python3
```
5. Run `source ~./bash_profile` to effictate the changes and reinitializate this jupyter notebook session.

In [1]:
!pip install pyspark

### 1.1 Product Counting Example 

To test your installation and start to get familiarized with Spark, we will follow an example that counts how many times the products of a toy dataset were purchased.

The main entry point to start programming with Spark is the [RDD API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD), an excellent Spark abstraction to work with the MapReduce framework.  RDD is a collection of elements partitioned across the nodes of the cluster that can operate in parallel. In other words, RDD is how Spark keeps your data ready to perform some function (e.g., a map or reduce function) in parallel. **Do not worry if this still sounds confusing, it will be clear once you start implementing**. However, it is part of this TP to study/consult the [Spark python API](https://spark.apache.org/docs/latest/api/python/) and learn how to use it. Some useful functions that the RDD API offers are:

1. **map**: return a new RDD by applying a function to each element of this RDD.
2. **flatMap**: return a new RDD by first applying a function to all elements of this RDD, and then flattening the results. **Should be used when each entry will yield more than one mapped element**
3. **reduce**: reduces the elements of this RDD using the specified commutative and associative binary operator.
4. **reduceByKey**: merge the values for each key using an associative and commutative reduce function
5. **groupByKey**: group the values for each key in the RDD into a single sequence
6. **collect**: return a list that contains all of the elements in this RDD. **Should not be used when working with a lot of data**
7. **sample**: return a sampled subset of this RDD
8. **count**: return the number of elements in this RDD.
9. **filter**: return a new RDD containing only the elements that satisfy a predicate.

In [2]:
from pyspark.sql import SparkSession

def map_to_product(row):
    """
    Map each transaction into a set of KEY-VALUE elements.
    The KEY is the word (product) itself and the VALUE is its number of apparitions.
    """
    products = row.transaction.split(';') # split products from the column transaction
    for p in products:
        yield (p, 1)

def reduce_product_by_key(value1, value2):
    "Reduce the mapped objects to unique words by merging (summing ) their values"
    return value1+value2

# Initializates a object of SparkSession class, main entry point to Spark's funcionalites
spark = SparkSession.builder.getOrCreate()
        
# Read a toy dataset
toy = spark.read.csv('toy.csv', header=True)
print("Toy dataset")
toy.show()

# Obtain a RDD object to call a map function
toy_rdd = toy.rdd
print("Toy dataframe as a RDD object (list of Row objects):\n\t", toy_rdd.collect())

# Map function to identify all products
toy_rdd = toy_rdd.flatMap(map_to_product)
print("\nMapped products:\n\t", toy_rdd.collect())

# Reduce function to merge values of elements that share the same KEY
toy_rdd = toy_rdd.reduceByKey(reduce_product_by_key)
print("\nReduced (merged) products:\n\t", toy_rdd.collect())

print("\nVisualizing as a dataframe:")
toy_rdd.toDF(["product", "count_product"]).show()

Toy dataset
+--------+-----------+
|order_id|transaction|
+--------+-----------+
|       1|      a;b;c|
|       2|      a;b;d|
|       3|        b;c|
|       4|        b;c|
+--------+-----------+

Toy dataframe as a RDD object (list of Row objects):
	 [Row(order_id='1', transaction='a;b;c'), Row(order_id='2', transaction='a;b;d'), Row(order_id='3', transaction='b;c'), Row(order_id='4', transaction='b;c')]

Mapped products:
	 [('a', 1), ('b', 1), ('c', 1), ('a', 1), ('b', 1), ('d', 1), ('b', 1), ('c', 1), ('b', 1), ('c', 1)]

Reduced (merged) products:
	 [('a', 2), ('b', 4), ('c', 3), ('d', 1)]

Visualizing as a dataframe:
+-------+-------------+
|product|count_product|
+-------+-------------+
|      a|            2|
|      b|            4|
|      c|            3|
|      d|            1|
+-------+-------------+



### 1.2 Working the Spark's Dataframe

In the example above, we briefly used a Spark's Dataframe class, but only to obtain an RDD object with ```toy.rdd``` and to print the data as a structured table with the ```show()``` function. However, [Dataframe](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#) is a big part of the current Spark release and is built upon the RDD API. It is a distributed collection of rows under named columns, the same as a table in a relational database. Spark's Dataframe works similarily as [Pandas'](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). In fact, we can export (obtain) a Spark's data frame to (from) a pandas' data frame with the function ```toPandas()``` (```spark.createDataFrame```).

A central functionality of the data frame is to profit from the [Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html#sql), a module that allows SQL queries over structured data. For example, the same 'product counting example' could have been implemented as a sequence of SQL operations over the data:  

In [3]:
import pyspark.sql.functions as f

# Creates a new column, products, with all products appering in each transaction
print('New column \'products\': exploding the transaction\'s products to a new row')
df_toy = toy.withColumn('products', f.explode(f.split(toy.transaction, ';')))
df_toy.show()

# Performs a select query and group rows by the product name, aggreagating by counting
print('Couting unique products:')
df_toy.select(df_toy.products)\
      .groupBy(df_toy.products)\
      .agg(f.count('products').alias('count_product'))\
      .sort('count_product', ascending=False)\
      .show()

New column 'products': exploding the transaction's products to a new row
+--------+-----------+--------+
|order_id|transaction|products|
+--------+-----------+--------+
|       1|      a;b;c|       a|
|       1|      a;b;c|       b|
|       1|      a;b;c|       c|
|       2|      a;b;d|       a|
|       2|      a;b;d|       b|
|       2|      a;b;d|       d|
|       3|        b;c|       b|
|       3|        b;c|       c|
|       4|        b;c|       b|
|       4|        b;c|       c|
+--------+-----------+--------+

Couting unique products:
+--------+-------------+
|products|count_product|
+--------+-------------+
|       b|            4|
|       c|            3|
|       a|            2|
|       d|            1|
+--------+-------------+



Also, the same SQL operations performed above could have been done with a traditional SQL language query as showing below:

In [4]:
# Creates a relational table TOY in the Spark session
df_toy.createOrReplaceTempView("TOY")

spark.sql("SELECT t.products, COUNT(t.products) AS product_count"
          " FROM TOY t"
          " GROUP BY t.products"
          " ORDER BY product_count DESC").show()

+--------+-------------+
|products|product_count|
+--------+-------------+
|       b|            4|
|       c|            3|
|       a|            2|
|       d|            1|
+--------+-------------+



These SQL concepts are being mentioned here because they will be useful to us during the TP, mainly in Section 3, to manipulate the supermarket data, which is structured in data frames. Then, if you are not familiar with SQL, it is recommended that you follow a [tutorial](https://www.w3schools.com/sql/) to understand the basics.

## 2. MBA Algorithm 
 The following sections explain how you should develop each step of the MapReduce algorithm for our supermarket application. Figure workflow.png illustrates each step of the algorithm.
 
### 2.1 Map to Patterns (10 points)
For a given a set of transactions (i.e., the rows of our toy dataset), each transaction must be **mapped** into a set of *purchase patterns* found within the transaction. Formally, these patterns are subsets of products that represent a group of items bought together. 
    
For the MapReduce framework, each pattern must be created as a *KEY-VALUE* element, where they KEY can take the form of a singleton, a pair or a trio of products that are present in the transaction. More precisely, for each transaction, the mapping function must generate all possible **unique** subsets of size **ONE, TWO or THREE**.  The VALUE associated with each KEY is the number of times that the KEY appeared in the transaction (if we assume that no product appears more than once in the transaction, this value is always equal to one). 

Now, implement the  **map_to_patterns** function that receives a transaction (a row from the data frame) and returns the patterns found in the transaction. The mapped elements are a tuple (KEY, VALUE), where KEY is also a tuple of product names. It is crucial to notice that, since each entry (transaction) of the map function will **yield** more than one KEY-VALUE element, a *flatMap* must be invoked for this step.

For the toy dataset, the expected output is similar to:

<pre style="align:center; border:1px solid black;font-size: 10pt; line-height: 1.1; height: auto; width: 18em; padding-left:5px">
<code>
+---------------+-----------+
|       patterns|occurrences|
+---------------+-----------+
|         ('a',)|          1|
|     ('a', 'b')|          1|
|('a', 'b', 'c')|          1|
|     ('a', 'c')|          1|
|         ('b',)|          1|
|     ('b', 'c')|          1|
|         ('c',)|          1|
|         ('a',)|          1|
|     ('a', 'b')|          1|
|('a', 'b', 'd')|          1|
|     ('a', 'd')|          1|
|         ('b',)|          1|
|     ('b', 'd')|          1|
|         ('d',)|          1|
|         ('b',)|          1|
|     ('b', 'c')|          1|
|         ('c',)|          1|
|         ('b',)|          1|
|     ('b', 'c')|          1|
|         ('c',)|          1|
+---------------+-----------+
</code>
</pre>

In [5]:
from itertools import combinations
def format_tuples(pattern):
    """
    Used for visualizition.
    Transforms tuples to a string since Dataframe does not support column of tuples with different sizes
    (a,b,c) -> '(a,b,c)'
    """
    return (str(pattern[0]), str(pattern[1]))

def map_to_patterns(row):
    transaction = row.transaction.split(';')
    combinations_table = []
    for x in range(1,len(transaction)+1):
        for combination in combinations(transaction,x):
            if combination != ' ':
                combinations_table.append((combination,1))
    return combinations_table
        
    
    """
    TODO
    """
    

toy_rdd = toy.rdd
patterns_rdd = toy_rdd.flatMap(map_to_patterns)

# Output as dataframe
patterns_rdd.map(format_tuples).toDF(['patterns', 'occurrences']).show()

+---------------+-----------+
|       patterns|occurrences|
+---------------+-----------+
|         ('a',)|          1|
|         ('b',)|          1|
|         ('c',)|          1|
|     ('a', 'b')|          1|
|     ('a', 'c')|          1|
|     ('b', 'c')|          1|
|('a', 'b', 'c')|          1|
|         ('a',)|          1|
|         ('b',)|          1|
|         ('d',)|          1|
|     ('a', 'b')|          1|
|     ('a', 'd')|          1|
|     ('b', 'd')|          1|
|('a', 'b', 'd')|          1|
|         ('b',)|          1|
|         ('c',)|          1|
|     ('b', 'c')|          1|
|         ('b',)|          1|
|         ('c',)|          1|
|     ('b', 'c')|          1|
+---------------+-----------+



### 2.2 Reduce patterns (2.5 points)
Once different CPUs processed the transactions, a **reduce** function must take place to combine identical KEYS (the subset of products) and compute the total number of its occurrences in the entire dataset. In other words, this reduce procedure must sum the *VALUE* of each identical KEY.

Create a **reduce_patterns** function below that must sum the VALUE of each pattern.
For the toy dataset, the expected output is:
<pre style="align:center; border:1px solid black;font-size: 10pt; line-height: 1.1; height: auto; width: 24em; padding-left:5px">
<code>
+---------------+--------------------+
|       patterns|combined_occurrences|
+---------------+--------------------+
|         ('a',)|                   2|
|     ('a', 'b')|                   2|
|('a', 'b', 'c')|                   1|
|     ('a', 'c')|                   1|
|         ('b',)|                   4|
|     ('b', 'c')|                   3|
|         ('c',)|                   3|
|('a', 'b', 'd')|                   1|
|     ('a', 'd')|                   1|
|     ('b', 'd')|                   1|
|         ('d',)|                   1|
+---------------+--------------------+
</code>
</pre>


In [6]:
import itertools
def reduce_patterns(patterns_rdd):
    combined_patterns_rdd = patterns_rdd.reduceByKey(reduce_product_by_key)
    return combined_patterns_rdd
combined_patterns_rdd = reduce_patterns(patterns_rdd)

# Output as dataframe
combined_patterns_rdd.map(format_tuples).toDF(['patterns', 'combined_occurrences']).show()

+---------------+--------------------+
|       patterns|combined_occurrences|
+---------------+--------------------+
|         ('a',)|                   2|
|         ('b',)|                   4|
|         ('c',)|                   3|
|     ('a', 'b')|                   2|
|     ('a', 'c')|                   1|
|     ('b', 'c')|                   3|
|('a', 'b', 'c')|                   1|
|         ('d',)|                   1|
|     ('a', 'd')|                   1|
|     ('b', 'd')|                   1|
|('a', 'b', 'd')|                   1|
+---------------+--------------------+



### 2.3 Map to subpatterns (15 points)
Next, another **map** function should be applied to generate subpatterns. Once again, the subpatterns are KEY-VALUE elements, where the KEY is a subset of products as well. However, creating the subpattern's KEY is a different procedure. This time, the idea is to break down the list of products of each pattern (pattern KEY), remove one product at a time, and yield the resulting list as the new subpattern KEY. 

For example, for a given pattern $P$ with three products, $p_1, p_2 $ and $p_3$, three new subpatterns KEYs are going to be created: (i) remove $p_1$ and yield ($p_2, p_3$); (ii) remove $p_2$ and yield ($p_1,p_3$); and (iii) remove $p_3$ and yield ($p_1,p_2$). 

Additionally, the subpattern's VALUE structure will also be different. Instead of just single integer value as we had in the patterns, this time a *tuple* should be created for the subpattern VALUE. This tuple contains the product that was removed when yielding the KEY and the number of times the pattern appeared. For the example above, the values should be ($p_1,v$), ($p_2,v$) and ($p_3,v$), respectively, where $v$ is the VALUE of the pattern. 

The idea behind subpatterns is to create **rules** such as: when the products of KEY were bought, the item present in the VALUE was also bought *v* times. Furthermore, each pattern should also yield a subpattern where the KEY is the same list of products of the pattern, but the VALUE is a tuple with a null product (None) and the number of times the pattern appeared. This element will be useful to keep track of how many times such a pattern was found and later will be used to compute the confidence value when generating the association rules. 

Now, implement the  **map_to_subpatterns** function that receives a pattern and yields all found subpatterns. Once again, each entry (pattern) will generate more than one KEY-VALUE element, then a flatMap function must be called.

For the toy dataset, the expected output is:

<pre style="align:center; border:1px solid black;font-size: 10pt; line-height: 1.1; height: auto; width: 17em; padding-left:5px">
<code>
+---------------+---------+
|    subpatterns|    rules|
+---------------+---------+
|         ('a',)|(None, 2)|
|     ('a', 'b')|(None, 2)|
|         ('b',)| ('a', 2)|
|         ('a',)| ('b', 2)|
|('a', 'b', 'c')|(None, 1)|
|     ('b', 'c')| ('a', 1)|
|     ('a', 'c')| ('b', 1)|
|     ('a', 'b')| ('c', 1)|
|     ('a', 'c')|(None, 1)|
|         ('c',)| ('a', 1)|
|         ('a',)| ('c', 1)|
|         ('b',)|(None, 4)|
|     ('b', 'c')|(None, 3)|
|         ('c',)| ('b', 3)|
|         ('b',)| ('c', 3)|
|         ('c',)|(None, 3)|
|('a', 'b', 'd')|(None, 1)|
|     ('b', 'd')| ('a', 1)|
|     ('a', 'd')| ('b', 1)|
|     ('a', 'b')| ('d', 1)|
|     ('a', 'd')|(None, 1)|
|         ('d',)| ('a', 1)|
|         ('a',)| ('d', 1)|
|     ('b', 'd')|(None, 1)|
|         ('d',)| ('b', 1)|
|         ('b',)| ('d', 1)|
|         ('d',)|(None, 1)|
+---------------+---------+
</code>
</pre>

In [14]:
from copy import deepcopy
def map_to_subpatterns(pattern):
    key = pattern[0]
    value = pattern[1]
    yield (key,(None,value))
    if len(key) > 1 :
        for combination in combinations(key, len(key)-1):
            for element in key :
                if element not in combination:
                    yield (combination,(element,value))



subpatterns_rdd = combined_patterns_rdd.flatMap(map_to_subpatterns)

# Output as dataframe
subpatterns_rdd.map(format_tuples).toDF(['subpatterns', 'rules']).show(100)

+---------------+---------+
|    subpatterns|    rules|
+---------------+---------+
|         ('a',)|(None, 2)|
|         ('b',)|(None, 4)|
|         ('c',)|(None, 3)|
|     ('a', 'b')|(None, 2)|
|         ('a',)| ('b', 2)|
|         ('b',)| ('a', 2)|
|     ('a', 'c')|(None, 1)|
|         ('a',)| ('c', 1)|
|         ('c',)| ('a', 1)|
|     ('b', 'c')|(None, 3)|
|         ('b',)| ('c', 3)|
|         ('c',)| ('b', 3)|
|('a', 'b', 'c')|(None, 1)|
|     ('a', 'b')| ('c', 1)|
|     ('a', 'c')| ('b', 1)|
|     ('b', 'c')| ('a', 1)|
|         ('d',)|(None, 1)|
|     ('a', 'd')|(None, 1)|
|         ('a',)| ('d', 1)|
|         ('d',)| ('a', 1)|
|     ('b', 'd')|(None, 1)|
|         ('b',)| ('d', 1)|
|         ('d',)| ('b', 1)|
|('a', 'b', 'd')|(None, 1)|
|     ('a', 'b')| ('d', 1)|
|     ('a', 'd')| ('b', 1)|
|     ('b', 'd')| ('a', 1)|
+---------------+---------+



### 2.4 Reduce Subpatterns (2.5 points)
Once again, a **reduce** function will be required to group all the subpatterns by their KEY. The objective of this reducing procedure is to create a list with all the **rules** that appeared by a KEY. Hence, the expected resulting of the reduce function is also a KEY-VALUE element, where the KEY is the subpattern's KEY, and the VALUE is a group containing all the VALUEs of the subpatterns that share the same KEY.

For the toy dataset, the expected output is:

<pre style="align:center; border:1px solid black;font-size: 10pt; line-height: 1.1; height: auto; width: 36em; padding-left:5px">
<code>
+---------------+-----------------------------------------+
|subpatterns    |combined_rules                           |
+---------------+-----------------------------------------+
|('a',)         |[(None, 2), ('b', 2), ('c', 1), ('d', 1)]|
|('a', 'b')     |[(None, 2), ('c', 1), ('d', 1)]          |
|('b',)         |[('a', 2), (None, 4), ('c', 3), ('d', 1)]|
|('a', 'b', 'c')|[(None, 1)]                              |
|('b', 'c')     |[('a', 1), (None, 3)]                    |
|('a', 'c')     |[('b', 1), (None, 1)]                    |
|('c',)         |[('a', 1), ('b', 3), (None, 3)]          |
|('a', 'b', 'd')|[(None, 1)]                              |
|('b', 'd')     |[('a', 1), (None, 1)]                    |
|('a', 'd')     |[('b', 1), (None, 1)]                    |
|('d',)         |[('a', 1), ('b', 1), (None, 1)]          |
+---------------+-----------------------------------------+
</code>
</pre>


In [15]:
combined_rules = subpatterns_rdd.groupByKey().mapValues(list)

# Output as dataframe
combined_rules.map(format_tuples).toDF(['subpatterns', 'combined_rules']).show(truncate=False)

+---------------+-----------------------------------------+
|subpatterns    |combined_rules                           |
+---------------+-----------------------------------------+
|('a',)         |[(None, 2), ('b', 2), ('c', 1), ('d', 1)]|
|('b',)         |[(None, 4), ('a', 2), ('c', 3), ('d', 1)]|
|('c',)         |[(None, 3), ('a', 1), ('b', 3)]          |
|('a', 'b')     |[(None, 2), ('c', 1), ('d', 1)]          |
|('a', 'c')     |[(None, 1), ('b', 1)]                    |
|('b', 'c')     |[(None, 3), ('a', 1)]                    |
|('a', 'b', 'c')|[(None, 1)]                              |
|('d',)         |[(None, 1), ('a', 1), ('b', 1)]          |
|('a', 'd')     |[(None, 1), ('b', 1)]                    |
|('b', 'd')     |[(None, 1), ('a', 1)]                    |
|('a', 'b', 'd')|[(None, 1)]                              |
+---------------+-----------------------------------------+



## 2.5. Map to Association Rules (15 points)
Finally, the last step of the algorithm is to create the association rules to perform the market basket analysis. The goal of this map function is to calculate the **confidence** level of buying a product, knowing that there is already a set of products in the basket. Thus, the KEY of the subpattern is the set of products placed in the basket and, for each product present in the list of rules, i.e., in the VALUE, the confidence can be calculated as:

\begin{align*}
\frac{\text{number of times the product was bought together with KEY }}{\text{number of times the KEY appeared}}
\end{align*}

For the example given in the Figure workflow, *coffee* was bought 20 times and, in 17 of them, *milk* was bought together. Then, the confidence level of buying *milk* knowing that *coffee* is in the basket is $\frac{17}{20} = 0.85$, which means that in 85% of the times the coffee was bought, milk was purchased as well.

Implement the **map_to_assoc_rules** function that calculates the confidence level for each subpattern.

For the toy dataset, the expected output is:
<pre style="align:center; border:1px solid black;font-size: 9pt; line-height: 1.1; height: auto; width: 35em; padding-left:5px">
<code>
+---------------+---------------------------------------+
|patterns       |association_rules                      |
+---------------+---------------------------------------+
|('a',)         |[('b', 1.0), ('c', 0.5), ('d', 0.5)]   |
|('a', 'b')     |[('c', 0.5), ('d', 0.5)]               |
|('b',)         |[('a', 0.5), ('c', 0.75), ('d', 0.25)] |
|('a', 'b', 'c')|[]                                     |
|('b', 'c')     |[('a', 0.3333333333333333)]            |
|('a', 'c')     |[('b', 1.0)]                           |
|('c',)         |[('a', 0.3333333333333333), ('b', 1.0)]|
|('a', 'b', 'd')|[]                                     |
|('b', 'd')     |[('a', 1.0)]                           |
|('a', 'd')     |[('b', 1.0)]                           |
|('d',)         |[('a', 1.0), ('b', 1.0)]               |
+---------------+---------------------------------------+
</code>
</pre>

In [37]:
def map_to_assoc_rules(rule):
    total_value = 0
    associated_rules = []
    new_associated_rules = []
    for element in rule[1]:
        if element[0] == None:
            total_value = element[1]
        else:
            associated_rules.append(element)
    for element in associated_rules:
        new_associated_rules.append((element[0],float(element[1])/total_value))
    assoc_rule = (rule[0],new_associated_rules)
    yield assoc_rule


assoc_rules = combined_rules.flatMap(map_to_assoc_rules)
# Output as dataframe
assoc_rules.map(format_tuples).toDF(['patterns', 'association_rules']).show(truncate=False)

+---------------+---------------------------------------+
|patterns       |association_rules                      |
+---------------+---------------------------------------+
|('a',)         |[('b', 1.0), ('c', 0.5), ('d', 0.5)]   |
|('b',)         |[('a', 0.5), ('c', 0.75), ('d', 0.25)] |
|('c',)         |[('a', 0.3333333333333333), ('b', 1.0)]|
|('a', 'b')     |[('c', 0.5), ('d', 0.5)]               |
|('a', 'c')     |[('b', 1.0)]                           |
|('b', 'c')     |[('a', 0.3333333333333333)]            |
|('a', 'b', 'c')|[]                                     |
|('d',)         |[('a', 1.0), ('b', 1.0)]               |
|('a', 'd')     |[('b', 1.0)]                           |
|('b', 'd')     |[('a', 1.0)]                           |
|('a', 'b', 'd')|[]                                     |
+---------------+---------------------------------------+



## 3. Instacart dataset

With your MBA algorithm ready to be used, now it is time to work on the real dataset. For this part of the TP, download the [instacart](https://www.instacart.com/datasets/grocery-shopping-2017) dataset and read its [description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) to understand how the dataset is structured. 

Before applying the developed algorithm on the instacart dataset, you must first filter the transactions to be in the same format defined by your algorithm (one transaction per row). To manipulate the data, we can use Spark's data frame and the SQL module presented in Section 1.

The following code cell uses the Spark SQL module to read the orders from the ``order_products__train.csv`` and the detailed information from ``orders.csv`` and ``products.csv`` to construct a data frame that contains a list of all products ever purchased by each user.

In [40]:
df_order_prod = spark.read.csv('instacart_2017_05_01/order_products__train.csv', header=True, sep=',', inferSchema=True)
print('order_products__train.csv')
df_order_prod.show(5)

df_orders = spark.read.csv('instacart_2017_05_01/orders.csv', header=True, sep=',', inferSchema=True)
print('orders.csv')
df_orders.show(5)

df_products = spark.read.csv('instacart_2017_05_01/products.csv', header=True, sep=',', inferSchema=True)
print('products.csv')
df_products.show(5)


"""
List of products ever purchased by each user
"""
# USING SQL
df_order_prod.createOrReplaceTempView("order_prod") # creates table 'order_prod'
df_orders.createOrReplaceTempView("orders") # creates table 'orders'
df_products.createOrReplaceTempView("products") # creates table 'products'
spark.conf.set("spark.sql.broadcastTimeout",  6000)
spark.sql('SELECT o.user_id, COLLECT_LIST(p.product_name) AS products' 
               ' FROM orders o '
               ' INNER JOIN order_prod op ON op.order_id = o.order_id'
               ' INNER JOIN products p    ON op.product_id = p.product_id'
               ' GROUP BY user_id ORDER BY o.user_id').show(5, truncate=80)


# USING DATAFRAME OPERATIONS
# df_orders.join(df_order_prod, df_order_prod.order_id == df_orders.order_id, 'inner')\
# .join(df_products, df_products.product_id == df_order_prod.product_id, 'inner')\
# .groupBy(df_orders.user_id).agg(f.collect_list(df_products.product_name).alias('products'))\
# .orderBy(df_orders.user_id).show(5, truncate=80)

order_products__train.csv
+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
+--------+----------+-----------------+---------+
only showing top 5 rows

orders.csv
+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                  null|
| 2398795|      1|   prior|           2|        3|                7|                  15.0|
|  473747|      1|   prior| 

Py4JJavaError: An error occurred while calling o1352.showString.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:146)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:387)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:140)
	at org.apache.spark.sql.executio

### 3.1 Business Insights (25 points) 

Now, you are the data scientist. Considering only the orders from ``order_products__train.csv``, use of Spark SQL module, performing with SQL or data frame, to answer the following questions:

1. What are the top 10 products which have the highest probability of being reordered (consider only products purchased at least 40 times)? 
2. What are the busiest shopping hours for each department?
    - Hint: plot a figure containing a time series for each department 
4. What is the size of the orders (basket size)? 
    - Hint: plot the basket size distribution 

Question 1

Question 2

In [43]:
spark.sql('WITH req AS (SELECT department_id, order_hour_of_day, COUNT(*) AS nb_apparition ' # prequery
          'FROM orders AS o, order_prod AS op, products AS p '
          'WHERE op.order_id = o.order_id AND op.product_id = p.product_id '
          'GROUP BY department_id, order_hour_of_day) '
          'SELECT req.department_id, req.order_hour_of_day, req.nb_apparition '#query
          'FROM ( '#subquery to find max_value
          'SELECT department_id as dep_id, MAX(req.nb_apparition) as max_apparition '
          'FROM req '
          'GROUP BY dep_id '
          ') AS max_req, req '
          'WHERE req.nb_apparition = max_req.max_apparition AND req.department_id = max_req.dep_id ' #join to choose the good result
          'ORDER BY CAST(req.department_id as int) ASC' #cast because data is string, and display in ASC
         ).show(30, truncate=80)

+-------------+-----------------+-------------+
|department_id|order_hour_of_day|nb_apparition|
+-------------+-----------------+-------------+
|            1|               14|         8718|
|            2|               14|          188|
|            3|               14|         4260|
|            4|               14|        35391|
|            5|               15|          620|
|            6|               13|         1086|
|            7|               10|         9790|
|            8|               11|          378|
|            9|               14|         3381|
|           10|               14|          133|
|           11|               14|         1801|
|           12|               14|         2670|
|           13|               14|         7127|
|           14|               14|         2533|
|           15|               11|         4094|
|           15|               13|         4094|
|           16|               14|        18599|
|           17|               10|       

Question 3

In [42]:
spark.sql('SELECT order_id, COUNT(*) ' 
               ' FROM order_prod '
               ' GROUP BY order_id'
               ).show(30, truncate=80)

+--------+--------+
|order_id|count(1)|
+--------+--------+
|    1342|       8|
|    1591|      31|
|    4519|       1|
|    4935|       1|
|    6357|       9|
|   10362|      36|
|   19204|      14|
|   29601|      17|
|   31035|       4|
|   40011|       6|
|   46266|      13|
|   51607|       6|
|   58797|       8|
|   61793|       6|
|   67089|      17|
|   70863|       5|
|   88674|       5|
|   91937|       5|
|   92317|      14|
|   99621|       8|
|  104688|      19|
|  106535|      10|
|  110904|       9|
|  120988|       5|
|  124647|       9|
|  124798|      11|
|  128389|      26|
|  131931|      44|
|  134748|       5|
|  141737|       1|
+--------+--------+
only showing top 30 rows



### 3.2 Run MBA for the training set (15 points)

Using the orders from the ``order_products__train.csv``, create a data frame where each row contain just one column, the transaction, with the list of purchased products. For this task, you do not have to query for the product names to run the MBA algorithm. You can assume that the product name is its id. This can accelerate the algorithm since the product names can be quite big.

You must report the time spent to perform this task.

In [ ]:
"""
TODO: create a query to create and sctruct the transactions
"""

In [ ]:
%%time
"""
TODO: run the MBA algorithm and show the first 5 association rules
"""

## 3.3 Run MBA for the whole dataset (15 points)

As you probably noticed, even for a not so large data set (the training file has only 131209 orders), the MBA algorithm is computationally expensive. For that reason, this time, we will repeat the process, but now using the Google Cloud Platform (GCP) to create a large computer cluster. All the instructions for creating a computing cluster with spark and how to submit a job will be explained in both sessions of the laboratory. In any case, you should read the instructions given in the ``Instruction_GCP.pdf``.

This time, we will work with the ``order_products__prior.csv`` file, which contains more than 3M orders.

**EXPECTED OUTPUT**

After you ran the MBA for the larger collection of orders, randomly select 5 products purchased in ``order_products__prior`` and print the association rules associated to those products when the product is alone in the basket. The output should be as described below:

**This time,  the products' name should be printed, not its ID. You can still perform the MBA using the IDs, but this last print must show the names.  
The output should be formatted in a table, each row containing the information of the association rules for one of the 10 products.**
       
Report the execution time.

In [ ]:
"""
TODO: create a query to create and sctruct the transactions from the order_products__prior.csv file
"""

In [ ]:
%%time
"""
TODO: run the MBA algorithm and print the requested output
"""